# Prochaine étape : circonscrire la portée de l'analyse

## Préambule

L'étude de [Hoppe et al. (2019)](https://www.science.org/doi/abs/10.1126/sciadv.aaw7238) a montré que le taux de financement plus faible des chercheur.e.s Afro-américain.e.s/noir.e.s (AA/B) s'explique par trois étapes du processus décisionnel de la part des évaluateur.trice.s :

> all three of the factors that underlie the funding gap—preference for some topics over others, assignment of poorer scores, and decision to discuss an application—revolve around decisions made by reviewers

De plus, l'étude de [Kozlowskia et al. (2019)](https://www.pnas.org/content/119/2/e2113067119) a montré que :

>  minoritized authors tend to publish in scientific disciplines and on research topics that reflect their gendered and racialized social identities

Autrement dit, que :

> there is a privilege of choice in scientific knowledge production, wherein research on a particular topic is influenced by scientist’s race and gender

On peut donc penser que des mécanismes similaires à ceux identifiés par Hoppe et al. (2019) et par [Magua et al. (2017)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5446598/) sont à l'oeuvre pour expliquer le sous-financement des femmes en recherche. Notre jeu de données ne nous permet cependant pas d'en étudier les causes sous-jacentes comme l'ont fait Hoppe et al. (2019), ni de mener une analyse intersectionnelle comme l'ont fait Kozlowskia et al. (2019). Il nous permet toutefois de faire l'état des lieux et d'explorer les questions ci-dessous.

## Exemples de questions exploratoires

* Pour étudier la concentration du financement :
    * Les femmes sont-elles moins financées que les hommes par les NIH?
    * Les femmes sont-elles financées aussi souvent que les hommes, i.e. quelle est la proportion de femmes vs hommes qui ont obtenu plus d'un financement? Combien en ont-elles/ils obtenus?
    * Quelle est la durée des projets financés pour les femmes vs les hommes?
* Et son évolution dans le temps :
    * Quelle est l'évolution du financement des femmes vs des hommes dans le temps? Le "funding gap" est-il stable, se réduit-il, augmente-t-il?
    * **RÉGRESSION :** Si le "funding gap" se réduit, dans combien d'années peut-on estimer que la parité sera atteinte si la tendance se maintient? (voir [Holman et al., 2018](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.2004956))
* Pour étudier l'effet de la collaboration scientifique :
    * Les femmes ont-elles davantage de co-PIs que les hommes ou l'inverse? Combien en ont-elles/ils?
    * Les équipes plus nombreuses sont-elles plus financées, plus souvent, plus longtemps?

## Autres questions pour aller plus loin...

* NLP : s'inspirer de la méthode de Hoppe et al. (2019) pour identifier des champs de recherche à partir du titre des projets et des termes associés
    * Quels sont les champs de recherche les plus et les moins financés?
    * Quels sont les champs de recherche où oeuvrent davantage les femmes vs les hommes? 
* Inclure les autres types de financement des NIH dans l'analyse à des fins de comparaison (p. ex. subvention d’infrastructure vs subvention de projet)

## Et en ajoutant d'autres données...

* D'autres données des NIH :
    * Les résumés des projets financés, pour améliorer l'identification des champs de recherche et mieux répondre aux questions ci-dessus
    * Les publications associées aux projets financés, pour étudier la concentration de la production scientifique :
        * À financement égal et en normalisant par champ de recherche, les femmes publient-elles autant que les hommes? (voir [Larivière et al., 2013](https://www.nature.com/articles/504211a))
* Des données externes :
    * Taille / prestige des universités d'attache des PIs contacts

## Problèmes, interrogations, bloquants, limitations

* Restreindre aux subventions de type [R01](https://grants.nih.gov/grants/funding/r01.htm) dans un premier temps, comme l'ont fait Magua et al. (2017)
* Attribuer un genre seulement aux PIs contacts lorsqu'il y a plusieurs PIs
* Exclure les sous-projets pour le moment :
    * Il y a une hiérarchie dans les données : des projets, des sous-projets. Pour un projet parent, le montant total est dans la colonne TOTAL_COST, mais pour un sous-projet, le montant sera dans TOTAL_COST_SUB_PROJECT et il n'y aura rien dans TOTAL_COST.
    * Pour voir si une ligne réfère à un projet parent ou à un sous-projet, il faut regarder dans la colonne SUBPROJECT_ID. Quand c'est vide, on parle d'un projet parent ou d'un projet qui n'a pas de sous-projets, et quand il y a un ID, il s'agit d'un sous-projet. Dans ce dernier cas, l'ID du projet parent se trouve dans la colonne FULL_PROJECT_NUMBER.
    * Pour les années plus récentes, on a aussi de nouvelles colonnes DIRECT_COST_AMT et INDIRECT_COST_AMT. En général, DIRECT_COST_AMT +  INDIRECT_COST_AMT = TOTAL_COST, mais parfois ce n'est pas le cas (par exemple, pour le APPLICATION_ID 8857469 qui n'a pourtant pas de sous-projet).

# Imports

In [2]:
import csv
import codecs
import pandas as pd
import itertools
import re
from datetime import datetime
from utils.csv_headers import *
import random

# Data Subsetting

### Approche 1

In [5]:
# check consistency of gender % for a specific period of time

def str_to_int(string):
    try:
        return int(string)
    except ValueError as a:
        return None


with codecs.open('content/enhanced-output.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')
    
    count_f = 0
    count_m = 0
    count_null = 0

    for row in dataset:
        FY_int = str_to_int(row[FY])
        if FY_int in range(2015, 2021):
            if row[CONTACT_PI_GENDER] == 'F':
                count_f += 1
            elif row[CONTACT_PI_GENDER] == 'M':
                count_m += 1
            else:
                count_null += 1

    print("count_f: ", count_f)
    print("count_m: ", count_m)
    print("count_null: ", count_null)
    print("sum: ", count_f + count_m + count_null)

count_f:  127228
count_m:  272183
count_null:  62022
sum:  461433


### Approche 2

In [14]:
# create a random subset with X% rows

SAMPLE_PERCENTAGE = 10

def str_to_int(string):
    try:
        return int(string)
    except ValueError as a:
        return None

with codecs.open('content/enhanced-output.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')

    with codecs.open('content/random_subset.csv', 'w', encoding='utf-8') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=dataset.fieldnames, dialect=csv.excel)
        writer.writeheader()

        for row in dataset:
            if random.randrange(0, 100) < SAMPLE_PERCENTAGE:
                 writer.writerow(row)

In [6]:
# QA: count number of successful gender assignment for the random-subset

with codecs.open('content/random_subset.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')
    
    count_f = 0
    count_m = 0
    count_null = 0

    for row in dataset:
        if row[CONTACT_PI_GENDER] == 'F':
            count_f += 1
        elif row[CONTACT_PI_GENDER] == 'M':
            count_m += 1
        else:
            count_null += 1

    print("count_f: ", count_f)
    print("count_m: ", count_m)
    print("count_null: ", count_null)
    print("sum: ", count_f + count_m + count_null)

count_f:  60638
count_m:  163014
count_null:  31053
sum:  254705


In [4]:
dataset = pd.read_csv('content/random_subset.csv')
dataset.head()

C:\Users\cpsar\.virtualenvs\NIH_project-34q57iAT\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,SERIAL_NUMBER,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT
0,3001993,A19,AH,1.0,NaN,NaN,7/1/1985,6/30/1986,NaN,A19AH000173,...,173,STC,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,3002019,A19,AH,1.0,NaN,NaN,7/1/1985,6/30/1986,NaN,A19AH000200,...,200,STC,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,3003935,D10,NU,1.0,NaN,NaN,2/1/1985,1/31/1988,NaN,D10NU024219,...,24219,PGR,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,3004120,D10,NU,5.0,NaN,NaN,8/1/1985,7/31/1986,NaN,D10NU025143,...,25143,PGR,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,3005815,D15,PE,5.0,NaN,NaN,7/1/1985,6/30/1986,NaN,D15PE014261,...,14261,STC,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN


### Approche 3

In [12]:
# compter la répartition F-H-NULL pour les subventions R01, pour tout le dataset entier

with codecs.open('content/enhanced-output.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')
    
    count_r01 = 0
    count_r01_f = 0
    count_r01_m = 0
    count_r01_null = 0
    
    for row in dataset:
        if row[ACTIVITY] == 'R01':
            count_r01 += 1
            if row[CONTACT_PI_GENDER] == 'F':
                count_r01_f += 1
            elif row[CONTACT_PI_GENDER] == 'M':
                count_r01_m += 1
            else:
                count_r01_null += 1

    print("count_r01_f: ", count_r01_f)
    print("count_r01_m: ", count_r01_m)
    print("count_r01_null: ", count_r01_null)
    print("sum_r01: ", count_r01_f + count_r01_m + count_r01_null) # should be the same as count_r01
    print("count_r01: ", count_r01)

count_r01_f:  197157
count_r01_m:  612168
count_r01_null:  85977
sum_r01:  895302
count_r01:  895302


# Subset Cleaning

In [5]:
# Removing lines and columns to reduce the scope of the project

subset_headers = [APPLICATION_ID,
                  CONTACT_PI_NAME, 
                  CONTACT_PI_GENDER, 
                  NB_PI_NAMES, 
                  ORG_COUNTRY, 
                  ORG_STATE, 
                  ORG_NAME, 
                  PROJECT_START, 
                  PROJECT_END, 
                  PROJECT_TITLE, 
                  PROJECT_TERMS, 
                  TOTAL_COST]

subset_headers_set = set(subset_headers) #on crée un set parce que c'est indexé et python va parcourir plus vite que lire une liste


with codecs.open('content/enhanced-output.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')

    with codecs.open('content/cleaned_subset.csv', 'w', encoding='utf-8') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=subset_headers, dialect=csv.excel)
        writer.writeheader()

        for row in dataset:
            if row[ACTIVITY] == 'R01' and \
                row[APPLICATION_TYPE] == '1' and \
                row[FY] == '2020' and \
                row[SUPPORT_YEAR] == '01' and \
                row[TOTAL_COST] != '' and (
                row[CONTACT_PI_GENDER] == 'F' or
                row[CONTACT_PI_GENDER] == 'M'
            ):
                subset_row = {}
                for key, value in row.items():
                    if key in subset_headers_set:
                        subset_row[key] = value
                writer.writerow(subset_row)

In [6]:
# QA on the cleaned subset

with codecs.open('content/cleaned_subset.csv', 'r', encoding='UTF-8') as csv_file_descriptor:
    dataset = csv.DictReader(csv_file_descriptor, delimiter=',', quotechar='"')
    
    count_r01 = 0
    count_r01_f = 0
    count_r01_m = 0
    count_r01_null = 0
    count_other_activities = 0
    
    for row in dataset:
        if row[ACTIVITY] == 'R01':
            count_r01 += 1
            if row[CONTACT_PI_GENDER] == 'F':
                count_r01_f += 1
            elif row[CONTACT_PI_GENDER] == 'M':
                count_r01_m += 1
            else:
                count_r01_null += 1
        else:
            count_other_activities += 1

    print("count_r01_f: ", count_r01_f)
    print("count_r01_m: ", count_r01_m)
    print("count_r01_null: ", count_r01_null)
    print("sum_r01: ", count_r01_f + count_r01_m + count_r01_null) # should be the same as count_r01
    print("count_r01: ", count_r01)
    print("count_other_activities: ", count_other_activities)

KeyError: 'ACTIVITY'